In [1]:
import tensorflow as tf
# Fixes bad convolution
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
@tf.function
def solve_line(s):
    stack = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
    stack = stack.write(0, 0)
    op_stack = tf.TensorArray(tf.string, size=0, dynamic_size=True, clear_after_read=False)
    op_stack = op_stack.write(0, '+')
    stack_ptr = 0
    
    def do_op(stack, stack_ptr, op_stack, e):
        cur_stack_val = stack.read(stack_ptr)
        last_op = op_stack.read(stack_ptr)
        if last_op == '+':
            stack = stack.write(stack_ptr, cur_stack_val + e)
        elif last_op == '*':
            stack = stack.write(stack_ptr, cur_stack_val * e)
        return stack
    
    for e in s:
        if e == '+':
            op_stack = op_stack.write(stack_ptr, '+')
        elif e == '*':
            op_stack = op_stack.write(stack_ptr, '*')
        elif e == '(':
            stack_ptr += 1
            stack = stack.write(stack_ptr, 0)
            op_stack = op_stack.write(stack_ptr, '+')
        elif e == ')':
            e = stack.read(stack_ptr)
            stack_ptr -= 1
            stack = do_op(stack, stack_ptr, op_stack, e)
        elif e == ' ':
            pass
        else:
            e = tf.strings.to_number(e, out_type=tf.float64)
            stack = do_op(stack, stack_ptr, op_stack, e)

    return stack.read(0)

@tf.function
def solve(s):
    s = tf.strings.split(s, '\n')
    s = tf.strings.bytes_split(s)

    return tf.reduce_sum(tf.map_fn(solve_line, s, fn_output_signature=tf.TensorSpec(shape=(), dtype=tf.float64)))

    
%timeit solve(tf.io.read_file('day18.txt'))

299 ms ± 9.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
#@tf.function
def solve_line(s):
    stack = tf.TensorArray(tf.float64, size=0, dynamic_size=True, clear_after_read=False)
    #stack = stack.write(0, 1)
    op_stack = tf.TensorArray(tf.string, size=0, dynamic_size=True, clear_after_read=False)
    #op_stack = op_stack.write(0, '*')
    stack_ptr = 0
    
    def do_op(stack, stack_ptr, op_stack, e):
        cur_stack_val = stack.read(stack_ptr)
        last_op = op_stack.read(stack_ptr)
        if last_op == '+':
            stack = stack.write(stack_ptr, cur_stack_val + e)
        elif last_op == '*':
            stack = stack.write(stack_ptr, cur_stack_val * e)
        return stack

    s = s[s != ' ']
    for i in tf.range(s.shape[0]):
        e = s[i]
        
        if i == s.shape[0]-1:
            lookahead = ' '
        else:
            lookahead = s[i+1]
        if e != ' ':
            print(stack.stack())
            print(e)
        if e == '+':
            op_stack = op_stack.write(stack_ptr, '+')
        elif e == '*':
            op_stack = op_stack.write(stack_ptr, '*')
        elif e == '(':
            stack_ptr += 1
            stack = stack.write(stack_ptr, 1)
            op_stack = op_stack.write(stack_ptr, '*')
        elif e == ')':
            e = stack.read(stack_ptr)
            stack_ptr -= 1
            stack = do_op(stack, stack_ptr, op_stack, e)
        elif e == ' ':
            pass
        else:
            if op_stack.size() == 0:
                op = ' '
            else:
                op = op_stack.read(stack_ptr)
            if op != '+' and lookahead == '+':
                stack_ptr += 1
                stack = stack.write(stack_ptr, 0)
                op_stack = op_stack.write(stack_ptr, '+')
                print(f'adding + stack, ptr {stack_ptr}')

            e = tf.strings.to_number(e, out_type=tf.float64)
            stack = do_op(stack, stack_ptr, op_stack, e)

            if op != '*' and lookahead == '*':
                print(f'decreasing stack ptr {stack_ptr}')
                e = stack.read(stack_ptr)
                stack_ptr -= 1
                stack = do_op(stack, stack_ptr, op_stack, e)
                stack = stack.write(stack_ptr+1, 0)

    return stack.stack(), op_stack.stack()

#@tf.function
def solve(s):
    s = tf.strings.split(s, '\n')
    print(s[0])
    s = tf.strings.bytes_split(s)
    return solve_line(s[0])
    return tf.reduce_sum(tf.map_fn(solve_line, s, fn_output_signature=tf.TensorSpec(shape=(), dtype=tf.float64)))

    
solve(tf.io.read_file('day18_test.txt'))

tf.Tensor(b'1 + 2 * 3 + 4 * 5 + 6', shape=(), dtype=string)
tf.Tensor([], shape=(0,), dtype=float64)
tf.Tensor(b'1', shape=(), dtype=string)
adding + stack, ptr 1
tf.Tensor([0. 1.], shape=(2,), dtype=float64)
tf.Tensor(b'+', shape=(), dtype=string)
tf.Tensor([0. 1.], shape=(2,), dtype=float64)
tf.Tensor(b'2', shape=(), dtype=string)
decreasing stack ptr 1
tf.Tensor([0. 0.], shape=(2,), dtype=float64)
tf.Tensor(b'*', shape=(), dtype=string)
tf.Tensor([0. 0.], shape=(2,), dtype=float64)
tf.Tensor(b'3', shape=(), dtype=string)
adding + stack, ptr 1
tf.Tensor([0. 3.], shape=(2,), dtype=float64)
tf.Tensor(b'+', shape=(), dtype=string)
tf.Tensor([0. 3.], shape=(2,), dtype=float64)
tf.Tensor(b'4', shape=(), dtype=string)
decreasing stack ptr 1
tf.Tensor([0. 0.], shape=(2,), dtype=float64)
tf.Tensor(b'*', shape=(), dtype=string)
tf.Tensor([0. 0.], shape=(2,), dtype=float64)
tf.Tensor(b'5', shape=(), dtype=string)
adding + stack, ptr 1
tf.Tensor([0. 5.], shape=(2,), dtype=float64)
tf.Tensor(b'+

(<tf.Tensor: shape=(2,), dtype=float64, numpy=array([ 0., 11.])>,
 <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'*', b'+'], dtype=object)>)

In [24]:
fib(10)

55

In [20]:
s = tf.io.read_file('day17.txt')

@tf.function
def solve2(s, n_cycles=6):
    s = tf.strings.split(s, '\n')
    s = tf.strings.bytes_split(s, 'UTF-8')
    s = s.to_tensor()
    s = tf.where(s == '.', 0.0, 1.0)
    s = s[None,None,...]
    s = tf.pad(s, 4*[2*[n_cycles]])
    s = s[None,...,None]
    
    n = 2*n_cycles + 1

    kernel = tf.ones(4*[3])
    kernel = tf.tensor_scatter_nd_update(kernel, [4*[1]], [0])[...,None,None]
    
    @tf.function(experimental_compile=True)
    def _solve(s):
        for i in range(n_cycles):
            convs = []
            # Thanks to https://github.com/funkey/conv4d/blob/master/conv4d.py#L27
            for j in range(3):
                convolution = tf.stack([
                    tf.nn.conv3d(s[:,i], kernel[j], strides=5*[1], padding='SAME')
                    for i in range(n)], axis=1)
                convs.append(convolution)
            
            conv0 = tf.concat((tf.zeros_like(convs[0][:,:1]), convs[0][:,:-1]), 1)
            conv1 = convs[1]
            conv2 = tf.concat((convs[2][:,1:], tf.zeros_like(convs[0][:,:1])), 1)
            
            convolution = conv0 + conv1 + conv2

            cells_to_deactivate = s * tf.cast((convolution != 2) & (convolution != 3), tf.float32)
            cells_to_activate = (1-s) * tf.cast(convolution == 3, tf.float32)

            s = s - cells_to_deactivate + cells_to_activate
    
        return tf.reduce_sum(s)

    return _solve(s)
    
%timeit solve2(s, n_cycles=6)
solve2(s, n_cycles=6)

13.6 ms ± 457 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<tf.Tensor: shape=(), dtype=float32, numpy=1504.0>

In [5]:
import datetime
# Set up logging.
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/day17/%s' % stamp

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.profiler.experimental.start(logdir)
solve2(s, n_cycles=6)
tf.profiler.experimental.stop()

In [19]:
# Set up logging.
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/day17_graph/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
solve2(s, n_cycles=6)
with writer.as_default():
    tf.summary.trace_export(
      name="conway_trace",
      step=0,
      profiler_outdir=logdir)

ProfilerNotRunningError: Cannot stop profiling. No profiler is running.